In [35]:
from neo4j import GraphDatabase
import json

In [36]:
with open('credentials.json') as json_file:
    credentials = json.load(json_file)

username = credentials['username']
pwd = credentials['password']

### NOTE ❣️

* BEFORE running this, still need to run `bin\neo4j console` to enable bolt on 127.0.0.1:7687
* When the queryyou wrote is wrong, the connection can be gone, and you need to restart the server through terminal

In [37]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=(username, pwd))

In [44]:
def delete_all(tx):
    result = tx.run("""match (n) detach delete n""")
    return result.single()

def create_entity(tx, entity_id, entity_name, entity_properties):
    cypher_query = """CREATE ("""+entity_id+""":"""+entity_name+ entity_properties+""")"""
    print(cypher_query)
    tx.run(cypher_query)
    
def display_all(tx):
    for record in tx.run("MATCH (c:Client)-[:PLACED]-(o)-[:CONTAINS]->(p) "
            "return c, o, p"):
        print(record)

Failed to write data to connection Address(host='localhost', port=7687) (Address(host='127.0.0.1', port=7687)); ("0; 'Underlying socket connection gone (_ssl.c:2053)'")
Failed to write data to connection Address(host='localhost', port=7687) (Address(host='127.0.0.1', port=7687)); ("0; 'Underlying socket connection gone (_ssl.c:2053)'")


In [40]:
def _create_and_return_greeting(tx, message):
    result = tx.run("CREATE (a:Greeting) "
                        "SET a.message = $message "
                        "RETURN a.message + ', from node ' + id(a)", message=message)
    return result.single()[0]

def print_greeting(message):
    with driver.session() as session:
        greeting = session.write_transaction(_create_and_return_greeting, message)
        print(greeting)
        
print_greeting('test')

test, from node 0


In [43]:
with driver.session() as session:
    output = session.write_transaction(delete_all)
    print(output)
#     session.write_transaction(create_entity, entity_id='Alice', entity_name='Client',
#         entity_properties = "{name:'Alice', ip: '1.1.1.1', shipping_address: 'a place', billing_address: 'a place'}")
    
#     session.read_transaction(display_all)

TypeError: 'NoneType' object is not subscriptable

In [18]:
def add_friend(tx, name, friend_name):
    tx.run("MERGE (a:Person {name: $name}) "
           "MERGE (a)-[:KNOWS]->(friend:Person {name: $friend_name})",
           name=name, friend_name=friend_name)

def print_friends(tx, name):
    for record in tx.run("MATCH (a:Person)-[:KNOWS]->(friend) WHERE a.name = $name "
                         "RETURN friend.name ORDER BY friend.name", name=name):
        print(record["friend.name"])

with driver.session() as session:
    session.write_transaction(add_friend, "Arthur", "Guinevere")
    session.write_transaction(add_friend, "Arthur", "Lancelot")
    session.write_transaction(add_friend, "Arthur", "Merlin")
    session.read_transaction(print_friends, "Arthur")

Guinevere
Lancelot
Merlin
